In [ ]:
#!pip install yfinance
#!pip install pytickersymbols

In [2]:
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytickersymbols import PyTickerSymbols
import statsmodels.api as sm
import seaborn as sns
from sklearn.linear_model import LinearRegression

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'SPY']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

#stocks = nasdaq_100 + sp_500
stocks = ['HPQ']

stocks = set(stocks) #Convierte la lista a Set, que lo único que hace es quitar elementos repetidos
stocks = list(stocks) #Reconvertir el set a list (para poder iterar sobre ella)

stocks.insert(0,'SPY')

periods = 60 #Especificar cuandos periodos (candles) se tomaran para cálculo de los Betas

In [4]:
len(stocks)

2

In [5]:
# df_precios = pd.DataFrame(yfinance.download('AAPL', period='10Y', interval='1mo')['Close'])
# df_precios = df[stocks]
# df_precios['Close'].dropna()

In [6]:
df_precios = pd.DataFrame([])

for i in range(len(stocks)):
  df_precios.insert(i, stocks[i], yfinance.download(stocks[i], period='10Y', interval='1mo')['Close'].dropna())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
#Dataframe de precios la fecha que se hace el rebalanceo (cada mes)
df_precios

,SPY,HPQ
Date,,
2011-05-01,134.899994,16.975477
2011-06-01,131.970001,16.530428
2011-07-01,129.330002,15.971844
2011-08-01,122.220001,11.821072
2011-09-01,113.150002,10.195277
...,...,...
2021-01-01,370.070007,24.340000
2021-02-01,380.359985,28.969999
2021-03-01,396.329987,31.750000


In [8]:
#Rendimientos mensuales
mtl_ret = df_precios.pct_change()#.dropna()
mtl_ret

,SPY,HPQ
Date,,
2011-05-01,NaN,NaN
2011-06-01,-0.021720,-0.026217
2011-07-01,-0.020005,-0.033791
2011-08-01,-0.054976,-0.259881
2011-09-01,-0.074210,-0.137534
...,...,...
2021-01-01,-0.010190,-0.010167
2021-02-01,0.027805,0.190222
2021-03-01,0.041987,0.095961


In [9]:
#Se crea el dataframe donde se guardaran los betas
totales = pd.DataFrame(index=mtl_ret.index).iloc[periods-1:-1]

In [10]:
#Sub dataframe con las columnas elegidas, quitando nan's y seleccionando solo el periodo a evaluar
sub = mtl_ret[['SPY', stocks[1]]].dropna().iloc[0:periods]

In [11]:
resultados = pd.DataFrame([])

for i in range(1,len(stocks)):
  lista_betas = []
  sub_col = mtl_ret[['SPY', stocks[i]]]#.dropna()
  for j in range(len(mtl_ret)-periods):
    sub_date = sub_col.iloc[j:periods+j].dropna()
    #print(sub_date)
    if len(sub_date) == periods and totales.index[j] == sub_date.index[-1]:
      X = sub_date['SPY']
      y = sub_date[stocks[i]]
      X_sm = sm.add_constant(X)

      regressor = LinearRegression(fit_intercept=False)
      regressor.fit(X_sm, y)
      beta = regressor.coef_[1]

      lista_betas.append(beta)
    else :
      lista_betas.append(0)

  totales.insert(i-1, stocks[i], lista_betas)

In [12]:
totales#.drop(index='2016-01-31')

,HPQ
Date,
2016-04-01,0.000000
2016-05-01,1.688365
2016-06-01,1.704100
2016-07-01,1.734769
2016-08-01,1.587031
...,...
2020-12-01,1.079792
2021-01-01,1.008820
2021-02-01,1.041572


In [13]:
portfolio_low = {}
portfolio_high = {}
PL_total = 0
trades_cerrados = 0

for i in range(1, len(totales)):
  row = totales.iloc[i] #Itera y obtiene los valores de cada fila
  row = row[row != 0].sort_values() #Obtiene los que son diferentes de cero y los ordena
  #Tickers de menor y mayor Beta para ese día
  low_beta_stock = row.index[:5]
  high_beta_stock = row.index[-5:]
  #Pasar la fecha a string para iloc
  date = totales.index[i].strftime('%Y-%m-%d')

  to_delete_low = []
  to_delete_high = []
  if len(portfolio_low) > 0:
    for j in portfolio_low.keys():
      if j not in low_beta_stock:
        pl_trade = df_precios[j].loc[date] - portfolio_low[j]
        PL_total += pl_trade
        to_delete_low.append(j)
        trades_cerrados += 1
        print(f'{date} Trade cerrado Long: {j}, {df_precios[j].loc[date]:.2f}-{portfolio_low[j]:.2f} = {PL_total:.2f}, {pl_trade/portfolio_low[j]:.2%}')
        # print(to_delete_low)
        # print(portfolio_low)
    for i in to_delete_low:
      del portfolio_low[i]

    for j in portfolio_high.keys():
      if j not in high_beta_stock:
        pl_trade = portfolio_high[j] - df_precios[j].loc[date]
        PL_total += pl_trade
        to_delete_high.append(j)
        trades_cerrados += 1
        print(f'{date} Trade cerrado Short: {j}, {portfolio_high[j]:.2f}-{df_precios[j].loc[date]:.2f} = {PL_total:.2f}, {pl_trade/portfolio_high[j]:.2%}')
        # print(to_delete_high)
        # print(portfolio_high)
    for i in to_delete_high:
      del portfolio_high[i]

  #Se agregan nuevos elementos a los portafolios (diccionarios)
  for j in range(len(low_beta_stock)):
    if low_beta_stock[j] not in portfolio_low.keys():
      portfolio_low[low_beta_stock[j]] = df_precios[low_beta_stock[j]].loc[date]
    
    if high_beta_stock[j] not in portfolio_high:
      portfolio_high[high_beta_stock[j]] = df_precios[high_beta_stock[j]].loc[date]

  #print(date, 'Portafolio bajo beta:', portfolio_low, 'Portafolio alto beta:', portfolio_high)


In [14]:
PL_total

0

In [15]:
trades_cerrados

0

In [16]:
# filter_df_precios  = df_precios[df_precios.index.isin(totales.index)]
# filter_df_precios

In [17]:
#Arroja el row donde la beta es el max/min o determinado valor
#df_resultados.loc[df_resultados[stocks[i]] == df_resultados[stocks[i]].max()]

In [18]:
#Ordenarlos para ver máximos y mínimos
#df_resultados.sort_values(by=1)